In [1]:
from sys import exit
import numpy as np
import pandas as pd
from os.path import dirname, join, isfile, isdir
from os import makedirs
from datetime import datetime, timedelta
from hashlib import sha256

from bulletin.data.tb_pacientes import TbPacientes
from bulletin.data.casos_confirmados import CasosConfirmados
from bulletin.commom.utils import Timer
from bulletin.commom import static
from bulletin.commom.normalize import *

pd.set_option('display.max_columns', None)

output = join("output","correcoes","casos_confirmados")

if not isdir(output):
    makedirs(output)

timer = Timer()

In [2]:
timer.reset()
casos_confirmados = CasosConfirmados()
casos_confirmados.update()
timer.stop()

Casos confirmados excluidos: 0
Obitos confirmados excluidos: 0


165.62779355049133

In [3]:
casos = casos_confirmados.get_casos()
casos.shape

(327299, 16)

In [4]:
casos = casos_confirmados.get_casos()

casos['ibge7'] = casos['ibge7'].apply(lambda x: str(x) if x != 99 else None)
casos['ibge'] = casos['ibge7'].apply(normalize_igbe)

casos_sem_ibge = casos.loc[casos['ibge'].isnull()].copy()
casos_sem_ibge = casos_sem_ibge.drop(columns=['ibge'])
casos_sem_ibge['mun_hash'] = casos_sem_ibge['mun_resid'].apply(normalize_hash)


municipios = static.municipios.copy()

casos_com_ibge = casos.loc[casos['ibge'].notnull()].copy()
casos_com_ibge = pd.merge(left=casos_com_ibge, right=municipios, how='left', on='ibge')

municipios['mun_hash'] = municipios['municipio_sesa'].apply(lambda x: normalize_hash(normalize_text(x)))
municipios_hash = municipios.drop_duplicates('mun_hash')
casos_sem_ibge = pd.merge(left=casos_sem_ibge, right=municipios_hash, how='left', on='mun_hash')
casos_sem_ibge = casos_sem_ibge.drop(columns=['mun_hash'])

casos_com_ibge = casos_com_ibge.append(casos_sem_ibge.loc[casos_sem_ibge['ibge'].notnull()]).sort_values('ordem')

casos_sem_ibge = casos_sem_ibge.loc[casos_sem_ibge['ibge'].isnull()].copy()
casos_sem_ibge = casos_sem_ibge.drop(columns=["cod_uf", "uf", "estado", "ibge", "municipio_sesa", "municipio_ibge"])
casos_sem_ibge['mun_hash'] = casos_sem_ibge['mun_resid'].apply(normalize_hash)

municipios['mun_hash'] = municipios['municipio_ibge'].apply(lambda x: normalize_hash(normalize_text(x)))
municipios_hash = municipios.drop_duplicates('mun_hash')

casos_sem_ibge = pd.merge(left=casos_sem_ibge, right=municipios_hash, how='left', on='mun_hash')
casos = casos_com_ibge.append(casos_sem_ibge).sort_values('ordem')

casos['ibge'] = casos['ibge'].fillna('99')
casos['ibge'] = casos['ibge'].apply(lambda x: str(x).zfill(2) if x != '99' else None)

casos = casos[['ordem','ibge7','ibge','nome','sexo','idade','mun_resid','uf_resid','municipio_sesa','municipio_ibge','uf','mun_atend','rs','laboratorio','dt_diag','comunicacao','is','hash','hash_less','hash_more']]

In [15]:
# casos.loc[casos['ibge'].isnull()].to_excel(join(output,'casos_sem_ibge.xlsx'))
# casos.loc[casos['ibge7'].isnull()].to_excel(join(output,'casos_sem_ibge7.xlsx'))
casos.loc[(casos['uf_resid'] == 'PR') & (casos['uf'] != 'PR'), 'uf_resid'] = casos.loc[(casos['uf_resid'] == 'PR') & (casos['uf'] != 'PR'), 'uf']
casos.loc[casos['uf']!='PR'] 

,ordem,ibge7,ibge,nome,sexo,idade,mun_resid,uf_resid,municipio_sesa,municipio_ibge,uf,mun_atend,rs,laboratorio,dt_diag,comunicacao,is,hash,hash_less,hash_more
0,6,None,355030,ITALO FERNANDO TROMBINI FILHO,M,28,SAO PAULO,SP,SAOPAULO,SAOPAULO,SP,CURITIBA,None,GENOPRIMER,2020-03-16,NaT,NaN,7a6331bb13dc13aeb8bf7e51304aad6abf81461c88cf3f...,2659a4ba6fc7cf45e2f1765001890bffe941003d9bd95d...,48e1eb28cbc4e32d8ebb702a2698085a0ff0dc2676805e...
1,69,None,530010,LUCIANO PUCHALSKI,M,59,BRASILIA,DF,BRASILIA,BRASILIA,DF,CURITIBA,None,LACEN,2020-03-23,2020-03-24,2020-03-13 00:00:00,a068784f769e79de5e23df41d6ef38c6b657a6cef49818...,f5ed75c0e3ecf244dbc8c4ca6496eec9910b16d3e11fdb...,04537895b26edce4b027ae7643974fae44c0e1e73baf70...
2,92,None,355030,JOAO HENRIQUE FONZAR BEGNINI,M,37,SAO PAULO,SP,SAOPAULO,SAOPAULO,SP,CAMBE,None,LACEN,2020-03-25,2020-03-25,2020-03-17 00:00:00,cd1ab1706dc270c0cc2d5a37d4a6921b75c6c0a165f219...,0555f630ab144fe80c4d415d0f1341508ba7420954362d...,c01929a5592970483132bf7aa349eae5121a1e2463bb7d...
3,165,None,330330,LUCINEIA ALVES DE ANHAIA,F,38,NITEROI,RJ,NITEROI,NITEROI,RJ,MANFRINOPOLIS,None,LACEN,2020-03-31,2020-03-31,2020-03-22 00:00:00,cc8b7fcba9ef0059612a861316829d24a7e492ddb83415...,ac369bd6ac91cc5bdca43c21bc03dde0ee04dfcc18209b...,5f8ab5b75a44563a7e22841b044b8fa02a44311bb720b9...
4,208,None,355030,PAULO FRANCISCO NETO,M,25,SAO PAULO,SP,SAOPAULO,SAOPAULO,SP,CURITIBA,None,LACEN,2020-03-31,2020-04-01,2020-03-12 00:00:00,1572142b2c40cc8cdbf30b27296bbb873ca7062e1ad5c1...,2e682174bba1bfe0ea7fb723710c51cc07b723ca78fbe2...,be414eec9caf1f82a4c4790c67408d99ef679fbf74f2ec...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2845,325450,None,353440,LUANA DA SILVA,F,28,OSASCO,SP,OSASCO,OSASCO,SP,LONDRINA,None,COVID-19 BIOLOGIA MOLECULAR,2020-12-11,2020-12-12,2020-12-06 00:00:00,51c008821816ce47f3dc4c7d3cc68b9f59304e83453591...,3f110580973df5da8cb06e35c81a4057ee9b15b87ed422...,e5399ea3d3e83bd69281c2a612b25cc65450b471d8ef91...
2846,325867,None,420530,MARIANA MARTINS MORAIS,F,84,FAXINAL DOS GUEDES,SC,FAXINALDOSGUEDES,FAXINALDOSGUEDES,SC,BOM SUCESSO,None,TESTE RAPIDO ANTIGENO (IMUNOCROMATOGRAFIA),2020-12-07,2020-12-12,2020-11-30 00:00:00,f3bdab11170737d02bc2437aaef91356dcddc112a0ed06...,ea0762fcbd642c251e962ea7d87bf93bdb2f8b5a636a49...,a922bed323d8123f759cac6b5e3e5bcbc81cb653bbb35b...
2847,326320,None,510180,PAULO ROBERTO CARRER,M,43,BARRA DO GARCAS,MT,BARRADOGARCAS,BARRADOGARCAS,MT,MEDIANEIRA,None,TESTE RAPIDO IGM/IGG NAO DISCRIMINATORIO (IMUN...,2020-12-11,2020-12-12,NaN,f446bc73c16bb9219678134bad66dbca6ad8922b5fa509...,a9b623c79e736591e546c0712b973dfcb9324360af58cc...,9ebe4c7b26b4be2baf38823b36d306f35ce91561e0fc5a...
2848,326591,None,292880,ROQUE PEREIRA FREITAS,M,40,SANTO ESTEVAO,BA,SANTOESTEVAO,SANTOESTEVAO,BA,TELEMACO BORBA,None,TESTE RAPIDO ANTIGENO (IMUNOCROMATOGRAFIA),2020-12-11,2020-12-12,NaN,cc90e818a0d64e3e4579b1907aae300ab2c8391c97a391...,f2a6019f37b940853394c54a0e7816274f145bec02e258...,8f5320a256a2732daf27c7954f1f616f6a84ad68bdbf76...


In [6]:
obitos = casos_confirmados.get_obitos()
obitos.shape

(6760, 7)

In [7]:
obitos.loc[obitos.duplicated('hash')].append(obitos.loc[obitos['hash'].isin(obitos['hash_less'])]).append(obitos.loc[obitos['hash'].isin(obitos['hash_more'])]).to_excel(join(output,'obitos_duplicados.xlsx'))

In [8]:
# obitos_nao_em_casos = obitos.loc[~obitos['hash'].isin(casos['hash'])]
# obitos_nao_em_casos.to_excel(join(output,'obitos_nao_em_casos.xlsx'))

# len(obitos_nao_em_casos)

# casos_idade_errada_less = obitos_nao_em_casos.loc[obitos['hash_less'].isin(casos['hash']), 'hash_less'].values.tolist()
# casos_idade_errada_more = obitos_nao_em_casos.loc[obitos['hash_more'].isin(casos['hash']), 'hash_more'].values.tolist()

# casos.loc[casos['hash'].isin(casos_idade_errada_less), 'idade'] -=1

# casos.loc[casos['hash'].isin(casos_idade_errada_less), 'hash'] = casos.loc[casos['hash'].isin(casos_idade_errada_less)].apply(lambda row: sha256(str.encode(normalize_hash(row['nome'])+str(row['idade'])+normalize_hash(row['mun_resid']))).hexdigest(), axis=1)
# casos.loc[casos['hash'].isin(casos_idade_errada_less), 'hash_less'] = casos.loc[casos['hash'].isin(casos_idade_errada_less)].apply(lambda row: sha256(str.encode(normalize_hash(row['nome'])+str(row['idade']-1)+normalize_hash(row['mun_resid']))).hexdigest(), axis=1)
# casos.loc[casos['hash'].isin(casos_idade_errada_less), 'hash_more'] = casos.loc[casos['hash'].isin(casos_idade_errada_less)].apply(lambda row: sha256(str.encode(normalize_hash(row['nome'])+str(row['idade']+1)+normalize_hash(row['mun_resid']))).hexdigest(), axis=1)


# casos.loc[casos['hash'].isin(casos_idade_errada_more), 'idade'] +=1
# casos.loc[casos['hash'].isin(casos_idade_errada_more), 'hash'] = casos.loc[casos['hash'].isin(casos_idade_errada_more)].apply(lambda row: sha256(str.encode(normalize_hash(row['nome'])+str(row['idade'])+normalize_hash(row['mun_resid']))).hexdigest(), axis=1)
# casos.loc[casos['hash'].isin(casos_idade_errada_more), 'hash_less'] = casos.loc[casos['hash'].isin(casos_idade_errada_more)].apply(lambda row: sha256(str.encode(normalize_hash(row['nome'])+str(row['idade']-1)+normalize_hash(row['mun_resid']))).hexdigest(), axis=1)
# casos.loc[casos['hash'].isin(casos_idade_errada_more), 'hash_more'] = casos.loc[casos['hash'].isin(casos_idade_errada_more)].apply(lambda row: sha256(str.encode(normalize_hash(row['nome'])+str(row['idade']+1)+normalize_hash(row['mun_resid']))).hexdigest(), axis=1)

# obitos.loc[~obitos['hash'].isin(casos['hash'])]

In [9]:
exit()
duplicados = casos.loc[casos.duplicated('hash')].index.tolist()
duplicados_less = casos.loc[casos['hash'].isin(casos['hash_less'])].index.tolist()
duplicados_more = casos.loc[casos['hash'].isin(casos['hash_more'])].index.tolist()
dropar = set(duplicados + duplicados_less + duplicados_more)
casos.loc[dropar].to_excel('casos_duplicados.xlsx')
print(len(dropar))
casos = casos.drop(index=dropar)
len(casos)

SystemExit: 

In [ ]:
duplicados = casos.loc[casos.duplicated('hash')].index.tolist()
duplicados_less = casos.loc[casos['hash'].isin(casos['hash_less'])].index.tolist()
duplicados_more = casos.loc[casos['hash'].isin(casos['hash_more'])].index.tolist()
dropar = set(duplicados + duplicados_less + duplicados_more)
casos.loc[dropar].to_excel('obitos_duplicados_casos_confirmados.xlsx')
len(dropar)

In [ ]:
writer = pd.ExcelWriter("novos_casos_confirmados.xlsx",
                        engine='xlsxwriter',
                        datetime_format='dd/mm/yyyy',
                        date_format='dd/mm/yyyy')

casos.sort_values('nome').to_excel(writer,index=None)

writer.save()
writer.close()